# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
Agrupados=pd.DataFrame(data.groupby(['CustomerID','ProductName']).sum().Quantity)
Agrupados

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
prod_customer=pd.pivot_table(data, values='Quantity', index='ProductName', columns='CustomerID', aggfunc='sum', fill_value=0)
prod_customer.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
squareform(pdist(prod_customer.T, 'euclidean'))

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

In [7]:
distancias = pd.DataFrame(1/(1 + squareform(pdist(prod_customer.T, 'euclidean'))), 
                         index=prod_customer.columns, columns=prod_customer.columns)

distancias.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [77]:
Top5=distancias[200].sort_values(ascending=False).head(6).reset_index()
Top5[1:]['CustomerID']

1    3317
2    1072
3    3535
4    1920
5    3909
Name: CustomerID, dtype: int64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [13]:
Agrupados.reset_index(inplace=True)

In [105]:
Seleccion=Agrupados[Agrupados.CustomerID.isin(Top5[1:]['CustomerID'])]
Seleccion.head()

,CustomerID,ProductName,Quantity
1082,1072,"Appetizer - Mini Egg Roll, Shrimp",1
1083,1072,Appetizer - Sausage Rolls,1
1084,1072,"Beans - Kidney, Canned",1
1085,1072,Beef - Montreal Smoked Brisket,1
1086,1072,Beef - Striploin Aa,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [28]:
Productos_recomend=pd.DataFrame(Seleccion.groupby('ProductName').sum()['Quantity'].sort_values(ascending=False)).reset_index()
Productos_recomend

,ProductName,Quantity
0,Soup - Campbells Bean Medley,4
1,Muffin - Carrot Individual Wrap,3
2,Bay Leaf,3
3,Pork - Kidney,3
4,"Pepper - Black, Whole",3
...,...,...
202,Kiwi,1
203,Knife Plastic - White,1
204,Lambcasing,1
205,"Lentils - Red, Dry",1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [89]:
Agrupados[Agrupados.CustomerID==200].head()

,CustomerID,ProductName,Quantity
59,200,Bacardi Breezer - Tropical,1
60,200,Bagel - Plain,1
61,200,Bar - Granola Trail Mix Fruit Nut,1
62,200,Beef - Ground Medium,1
63,200,Beef - Montreal Smoked Brisket,1


In [55]:
result = pd.merge(Productos_recomend, Agrupados[Agrupados.CustomerID==200], on="ProductName", how='left')
top_recomendaciones=result[result.CustomerID.isnull()].head(5)
list(top_recomendaciones['ProductName'])
#merge ambos df, manteniendo todos los nombres de los productos y luego filtrar aquellas filas donde está vacío el valor de Customer ID

['Soup - Campbells Bean Medley',
 'Muffin - Carrot Individual Wrap',
 'Bay Leaf',
 'Pork - Kidney',
 'Wanton Wrap']

In [56]:
dic={}

In [58]:
dic[200]=list(top_recomendaciones['ProductName'])
dic

{200: ['Soup - Campbells Bean Medley',
  'Muffin - Carrot Individual Wrap',
  'Bay Leaf',
  'Pork - Kidney',
  'Wanton Wrap']}

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [83]:
dic_recomendaciones={}
Custom_ID=list(data.CustomerID.unique())

In [100]:
for customer in Custom_ID:
    Top5_customer=distancias[customer].sort_values(ascending=False).head(6).reset_index()#top 5 customers cercanos al cliente
    
    customers_cercanos=Agrupados[Agrupados.CustomerID.isin(Top5_customer[1:]['CustomerID'])]#df productos de customers cercanos
    
    Productos_recomend=pd.DataFrame(customers_cercanos.groupby('ProductName').sum()['Quantity'].sort_values(ascending=False)).reset_index()#agrupa los productos más comprados para recomendar
    
    merge_df = pd.merge(Productos_recomend, Agrupados[Agrupados.CustomerID==customer], on="ProductName", how='left')#une df de productos recomendados con df de productos del cliente
    
    top_recomendaciones=list(merge_df[merge_df.CustomerID.isnull()].head(5)['ProductName'])#selecciona el top5 de productos más vendidos y no comprados por el cliente para recomendar
    
    dic_recomendaciones[customer]=top_recomendaciones

In [103]:
dic_recomendaciones[200]

['Soup - Campbells Bean Medley',
 'Muffin - Carrot Individual Wrap',
 'Bay Leaf',
 'Pork - Kidney',
 'Wanton Wrap']

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [115]:
DF=pd.DataFrame.from_dict(dic_recomendaciones,orient='index').reset_index()
DF.rename(columns={'index':'CustomerID',0:'recomendacion1',1: 'recomendacion2', 2:'recomendacion3', 3:'recomendacion4', 4:'recomendacion5'})

,CustomerID,recomendacion1,recomendacion2,recomendacion3,recomendacion4,recomendacion5
0,61288,Jagermeister,Chicken - Soup Base,Lime Cordial - Roses,Macaroons - Two Bite Choc,Flavouring - Orange
1,77352,Wine - Valpolicella Masi,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt
2,40094,Bread - Italian Corn Meal Poly,Puree - Mocha,Tuna - Salad Premix,Sherry - Dry,Beer - Sleemans Cream Ale
3,23548,Squid - Tubes / Tenticles 10/20,Sprouts - Baby Pea Tendrils,Flavouring - Orange,Bread - Raisin Walnut Oval,Banana Turning
4,78981,Vanilla Beans,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Cinnamon Buns Sticky,Pop Shoppe Cream Soda
...,...,...,...,...,...,...
995,49005,"Pasta - Penne, Rigate, Dry",Browning Caramel Glace,Tea - Decaf Lipton,"Garlic - Primerba, Paste","Wine - Red, Harrow Estates, Cab"
996,41286,Veal - Inside,"Thyme - Lemon, Fresh",Wonton Wrappers,"Veal - Inside, Choice",Bread Crumbs - Japanese Style
997,85878,Olives - Kalamata,Wine - White Cab Sauv.on,Veal - Eye Of Round,Bread - Calabrese Baguette,Cheese - Wine
998,68506,"Veal - Brisket, Provimi,bnls",Beef - Ground Medium,"Rosemary - Primerba, Paste",Scallops - 10/20,"Oregano - Dry, Rubbed"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [116]:
distancias_2 = pd.DataFrame(1/(1 + squareform(pdist(prod_customer.T, 'cosine'))), 
                         index=prod_customer.columns, columns=prod_customer.columns)

distancias_2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.530003,0.563492,0.543546,0.542393,0.541054,0.522016,0.530783,0.533096,0.535293,...,0.530306,0.517987,0.527662,0.511471,0.534181,0.513809,0.528795,0.520208,0.529481,0.527142
200,0.530003,1.000000,0.537166,0.535098,0.521226,0.519193,0.556945,0.516729,0.536350,0.541451,...,0.545224,0.533067,0.555246,0.543554,0.530308,0.521925,0.573177,0.537284,0.526167,0.527280
264,0.563492,0.537166,1.000000,0.531963,0.550168,0.525368,0.538967,0.514932,0.537046,0.524193,...,0.546880,0.525550,0.535062,0.531806,0.553085,0.560225,0.548538,0.538340,0.554929,0.527142
356,0.543546,0.535098,0.531963,1.000000,0.525982,0.523986,0.536792,0.540890,0.516901,0.519482,...,0.536321,0.520565,0.533119,0.534066,0.528874,0.530406,0.520138,0.536202,0.544727,0.535974
412,0.542393,0.521226,0.550168,0.525982,1.000000,0.534073,0.542308,0.547743,0.530459,0.543050,...,0.527903,0.565369,0.528847,0.529278,0.541674,0.536686,0.519628,0.538710,0.546943,0.552636


In [117]:
dic_recomendaciones_2={}

for customer in Custom_ID:
    Top5_customer=distancias_2[customer].sort_values(ascending=False).head(6).reset_index()#top 5 customers cercanos al cliente
    
    customers_cercanos=Agrupados[Agrupados.CustomerID.isin(Top5_customer[1:]['CustomerID'])]#df productos de customers cercanos
    
    Productos_recomend=pd.DataFrame(customers_cercanos.groupby('ProductName').sum()['Quantity'].sort_values(ascending=False)).reset_index()#agrupa los productos más comprados para recomendar
    
    merge_df = pd.merge(Productos_recomend, Agrupados[Agrupados.CustomerID==customer], on="ProductName", how='left')#une df de productos recomendados con df de productos del cliente
    
    top_recomendaciones=list(merge_df[merge_df.CustomerID.isnull()].head(5)['ProductName'])#selecciona el top5 de productos más vendidos y no comprados por el cliente para recomendar
    
    dic_recomendaciones_2[customer]=top_recomendaciones

In [118]:
dic_recomendaciones_2[200]

['Longos - Grilled Salmon With Bbq',
 'Snapple Lemon Tea',
 'General Purpose Trigger',
 'Thyme - Lemon, Fresh',
 'Tomatoes Tear Drop']

In [119]:
DF_coseno=pd.DataFrame.from_dict(dic_recomendaciones_2,orient='index').reset_index()
DF_coseno.rename(columns={'index':'CustomerID',0:'recomendacion1',1: 'recomendacion2', 2:'recomendacion3', 3:'recomendacion4', 4:'recomendacion5'})

,CustomerID,recomendacion1,recomendacion2,recomendacion3,recomendacion4,recomendacion5
0,61288,Jagermeister,Wine - Two Oceans Cabernet,Tofu - Firm,Appetizer - Sausage Rolls,Dc - Frozen Momji
1,77352,Bread - Rye,Fondant - Icing,Lettuce - Frisee,Bread - French Baquette,Pork - Kidney
2,40094,Juice - V8 Splash,Beef - Inside Round,"Bread - Roll, Soft White Round",Truffle Cups - Brown,Cake - Mini Cheesecake
3,23548,Juice - Orange,Bagel - Plain,Olives - Stuffed,Veal - Sweetbread,Cinnamon Buns Sticky
4,78981,Wine - Blue Nun Qualitatswein,Cookies - Assorted,Cheese - Cottage Cheese,Baking Powder,Quiche Assorted
...,...,...,...,...,...,...
995,49005,Squid - Tubes / Tenticles 10/20,"Pepper - White, Ground",Soup Knorr Chili With Beans,Cheese - Bocconcini,Barramundi
996,41286,"Hickory Smoke, Liquid","Salsify, Organic",Dc Hikiage Hira Huba,Oil - Shortening - All - Purpose,Banana - Leaves
997,85878,Bread - Calabrese Baguette,French Pastry - Mini Chocolate,Cheese - Parmesan Grated,Foam Dinner Plate,Cheese - Mix
998,68506,Flavouring - Orange,Longos - Chicken Wings,"Cheese - Brie, Triple Creme",Tia Maria,Fondant - Icing
